<a href="https://colab.research.google.com/github/luasm17/LLM_as_a_judge/blob/main/prometheus_zero_shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalar dependencias + Imports
!pip install -q -U "transformers>=4.51.0" accelerate safetensors

import os
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 100.7 MB/s eta 0:00:00


In [2]:
# Cargamos el archivo CSV que se sube a Colab.
# Debe contener las columnas input_modelo y output_modelo.
# Cada fila representa un par input-output del corrector.

file_path = "/content/frases_corrector.csv"
df = pd.read_csv(file_path, encoding='latin1', delimiter=';')

In [3]:
# Token HF
HF_TOKEN = os.environ.get("HF_TOKEN", None)

In [4]:
# Cargar modelo e tokenizer (M-Prometheus-7B)
# LLM-as-a-Judge binario para concordancia de número en galego

MODEL_ID = "Unbabel/M-Prometheus-7B"

print("🔄 Cargando tokenizer...")
if HF_TOKEN:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
else:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

print("🔄 Cargando modelo (pode tardar uns minutos)...")
if HF_TOKEN:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        token=HF_TOKEN,
        device_map="auto",
        torch_dtype="auto"
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        device_map="auto",
        torch_dtype="auto"
    )

model.eval()
print("✅ Modelo cargado correctamente")

🔄 Cargando tokenizer...


config.json:   0%|          | 0.00/790 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

✅ Modelo cargado correctamente


In [6]:
results = []

In [ ]:
for i, row in df.iterrows():

    input_text = row["input_modelo"]
    output_text = row["output_modelo"]

    prompt = f"""
Es un LLM-as-a-judge que vai avaliar un modelo de corrección gramatical (GEC) en galego.
En concreto, a túa tarefa é a de avaliar a saída dun corrector gramatical e decidir se o modelo GEC corrixiu correctamente o erro de concordancia
de número da frase inicial (é dicir, singular/plural entre determinante, substantivo, adxectivo ou verbo).

Recibirás sempre:
  input_corrector: a frase orixinal que se lle pasou ao corrector
  output_corrector: a frase devolta polo corrector

Tes que avaliar respondendo exactamente aos seguintes catro elementos e seguindo este formato, sen ningún texto adicional:
  input_corrector: "<oración de entrada do corrector>"
  output_corrector: "<oración xa avaliada polo corrector>"
  tag: <0 ou 1>
  explanation: "<explicación breve e precisa en galego do motivo polo que se escolleu a etiqueta 0 ou a etiqueta 1>"

Os criterios que debes seguir para avaliar son:
  tag = 1: a saída do corrector non é correcta (segue habendo erro de concordancia de número no output_corrector).
  tag = 0: a saída do corrector é correcta (non hai erro de concordancia de número no output_corrector).

As restricións que debes respetar son:
  Non debes, baixo ningún concepto, corrixir ou modificar de ningunha forma o input nin o output do corrector.
  Tes que limitarte exclusivamente a decidir se o erro de concordancia de número foi corrixido polo modelo GEC ou non.
  Non debes avaliar outros tipos de erros.

Agora, avalía o seguinte caso:
input_corrector: "{input_text}"
output_corrector: "{output_text}"
"""

    # Tokenizo el prompt y lo envío al mismo dispositivo que el modelo
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generación determinista
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=300,
            temperature=0,
            do_sample=False
        )

    # Recorto los tokens correspondientes al prompt para quedarme solo con lo generado
    generated_tokens = outputs[0][inputs["input_ids"].shape[-1]:]

    decoded = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    # Guardo el resultado
    results.append(decoded)

    # Lo imprimo en pantalla
    print("\nResultado del ejemplo", i + 1)
    print(decoded)
    print()


Resultado del ejemplo 1
  tag: 0
  explanation: "O corrector non detectou o erro de concordancia entre 'eleccións' e 'elixir'. A frase está correcta porque 'eleccións' é plural e 'elixir' tamén é plural, polo que hai concordancia de número."
```json
{
  "input_corrector": "As eleccións para escoller ao novo presidente da comunidade non tiveron lugar por mor do mal ambiente no condominio.",
  "output_corrector": "As eleccións para elixir o novo presidente da comunidade non tiveron lugar por mor do mal ambiente no condominio.",
  "tag": 0,
  "explanation": "O corrector non detectou o erro de concordancia entre 'eleccións' e 'elixir'. A frase está correcta porque 'eleccións' é plural e 'elixir' tamén é plural, polo que hai concordancia de número."
}
```


Resultado del ejemplo 2
tag: 0
explanation: "Non hai erro de concordancia de número no output_corrector. O verbo 'presentou' está en concordancia coa súmula 'grupo de estudantes', que é singular."
```json
{
  "input_corrector": "O grupo